# Corporación Favorita Grocery Sales Forecasting
## w01_d05_EDA_context_export.ipynb

**Author:** Alberto Diaz Durana  
**Date:** November 2025  
**Purpose:** Analyze holidays, promotions, perishables; export final analysis-ready dataset

---

## Objectives

This notebook accomplishes the following:

- Analyze holiday impact on sales (by holiday type, pre/post effects)
- Measure promotion effectiveness (sales lift, frequency)
- Investigate promotion × holiday interactions
- Compare perishable vs non-perishable patterns
- Identify high-volatility items (waste risk)
- Analyze oil price correlation
- Export final cleaned dataset with all features
- Create Week 1 summary report

---

## Business Context

**Why external factors matter:**

Understanding holidays, promotions, and product characteristics enables:
- Optimal promotional timing (avoid/leverage holidays)
- Inventory risk management (perishables require higher accuracy)
- Resource planning (holiday staffing, promotional support)
- Waste reduction (identify high-volatility perishables)

**Deliverables:**
- Holiday analysis report with sales lift by type
- Promotion effectiveness metrics (ROI quantification)
- Perishable waste indicators
- Final dataset: guayas_prepared.csv (analysis-ready)
- Week 1 summary report

---

## Input Dependencies

From Day 4:
- Clean dataset with temporal + rolling features (300K rows, 26 columns)
- Store and item metadata merged

From raw data:
- holidays_events.csv (350 holiday records)
- oil.csv (1,218 oil price records)

---

## 1. Holiday Effects Analysis

**Objective:** Quantify impact of holidays on sales patterns

**Activities:**
- Merge holidays_events.csv with main dataset
- Calculate proximity to holidays (days before/after)
- Compare sales on holidays vs normal days
- Identify holiday types with strongest impact
- Analyze transferred holidays behavior
- Create holiday visualization (bar chart by type)

**Expected output:** 
- Holiday proximity features (days_to_holiday, days_after_holiday)
- Sales lift metrics by holiday type
- Visualization showing holiday vs non-holiday sales
- Business recommendations for holiday inventory

⚠️ **Note:** Computation time ~4-5 minutes due to date proximity calculations.

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import scipy
from scipy import stats

# Configure environment
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('seaborn-v0_8-darkgrid')

print("Package Versions:")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print(f"  matplotlib: {matplotlib.__version__}")
print(f"  seaborn: {sns.__version__}")
print(f"  scipy: {scipy.__version__}")
print("\nOK - Libraries imported")

In [ ]:
# Determine paths
current_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
project_root = current_dir.parent if current_dir.name == 'notebooks' else current_dir

# Define path constants
DATA_RAW = project_root / 'data' / 'raw'
DATA_PROCESSED = project_root / 'data' / 'processed'
OUTPUTS = project_root / 'outputs' / 'figures' / 'eda'

# Verify paths
assert DATA_RAW.exists(), f"ERROR - Path not found: {DATA_RAW}"
assert DATA_PROCESSED.exists(), f"ERROR - Path not found: {DATA_PROCESSED}"
assert OUTPUTS.exists(), f"ERROR - Path not found: {OUTPUTS}"

print("OK - Paths validated:")
print(f"  Project root: {project_root.resolve()}")
print(f"  DATA_RAW: {DATA_RAW.resolve()}")
print(f"  DATA_PROCESSED: {DATA_PROCESSED.resolve()}")
print(f"  OUTPUTS: {OUTPUTS.resolve()}")

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print(f"\nRandom seed: {RANDOM_SEED}")

In [ ]:
# Load dataset from Day 4 (need to reapply transformations)
print("Loading dataset and reapplying Day 3-4 transformations...")

df = pd.read_pickle(DATA_PROCESSED / 'guayas_sample_300k.pkl')

# Quick transformations from Days 3-4
df['date'] = pd.to_datetime(df['date'])
df['onpromotion'] = df['onpromotion'].fillna(0.0)

# Merge store metadata
df_stores = pd.read_csv(DATA_RAW / 'stores.csv')
df = df.merge(df_stores[['store_nbr', 'city', 'state', 'type', 'cluster']], 
              on='store_nbr', how='left')

# Create temporal features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

print(f"OK - Dataset ready")
print(f"  Shape: {df.shape}")
print(f"  Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"  Missing values: {df.isnull().sum().sum()}")

In [ ]:
# Load and explore holidays data
print("Holiday Impact Analysis")
print("=" * 70)

print("\nLoading holidays_events.csv...")
df_holidays = pd.read_csv(DATA_RAW / 'holidays_events.csv')

print(f"OK - Holidays loaded")
print(f"  Total holiday records: {len(df_holidays)}")
print(f"  Columns: {list(df_holidays.columns)}")

print("\nFirst 10 rows:")
print(df_holidays.head(10))

print("\nHoliday types:")
print(df_holidays['type'].value_counts())

print("\nLocale distribution:")
print(df_holidays['locale'].value_counts())

print("\nLocale names (sample):")
print(df_holidays['locale_name'].value_counts().head(10))

# Filter to National and Guayas holidays
print("\nFiltering to National and Guayas (regional/local) holidays...")
df_holidays_filtered = df_holidays[
    (df_holidays['locale'] == 'National') | 
    (df_holidays['locale_name'] == 'Guayas')
].copy()

print(f"  Filtered holidays: {len(df_holidays_filtered)} (from {len(df_holidays)})")

# Convert date to datetime
df_holidays_filtered['date'] = pd.to_datetime(df_holidays_filtered['date'])

print(f"\nDate range: {df_holidays_filtered['date'].min().date()} to {df_holidays_filtered['date'].max().date()}")

In [ ]:
# Merge holidays with sales data (CORRECTED)
print("\nMerging holidays with sales data (CORRECTED)...")

# Start fresh - reload to avoid duplicate columns
df = pd.read_pickle(DATA_PROCESSED / 'guayas_sample_300k.pkl')
df['date'] = pd.to_datetime(df['date'])
df['onpromotion'] = df['onpromotion'].fillna(0.0)

# Merge store metadata
df_stores = pd.read_csv(DATA_RAW / 'stores.csv')
df = df.merge(df_stores[['store_nbr', 'city', 'state', 'type', 'cluster']], 
              on='store_nbr', how='left')

# Create temporal features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Create holiday flag
df['is_holiday'] = df['date'].isin(df_holidays_filtered['date']).astype(int)

# Merge holiday details (avoid column name conflicts)
df_holidays_merge = df_holidays_filtered[['date', 'type', 'description']].rename(
    columns={'type': 'holiday_type', 'description': 'holiday_name'}
)

df = df.merge(df_holidays_merge, on='date', how='left')

print(f"OK - Merge complete")
print(f"  Dataset shape: {df.shape}")
print(f"  Columns: {len(df.columns)}")

# CORRECTED CALCULATION: Compare daily averages
print("\nNon-holiday vs Holiday comparison (CORRECTED - Daily Averages):")
non_holiday_sales = df[df['is_holiday'] == 0]['unit_sales'].sum()
holiday_sales = df[df['is_holiday'] == 1]['unit_sales'].sum()
non_holiday_days = df[df['is_holiday'] == 0]['date'].nunique()
holiday_days = df[df['is_holiday'] == 1]['date'].nunique()

non_holiday_avg = non_holiday_sales / non_holiday_days
holiday_avg = holiday_sales / holiday_days

print(f"  Non-holiday: {non_holiday_sales:,.0f} units / {non_holiday_days} days = {non_holiday_avg:,.1f} units/day")
print(f"  Holiday: {holiday_sales:,.0f} units / {holiday_days} days = {holiday_avg:,.1f} units/day")
print(f"  Holiday lift: {((holiday_avg / non_holiday_avg) - 1) * 100:+.1f}%")

print("\nHoliday type distribution:")
print(df[df['is_holiday'] == 1]['holiday_type'].value_counts())

In [ ]:
# Analyze sales by holiday type
print("\nSales Analysis by Holiday Type:")
print("=" * 70)

# Calculate average daily sales by holiday type
holiday_type_analysis = df[df['is_holiday'] == 1].groupby('holiday_type').agg({
    'unit_sales': 'sum',
    'date': 'nunique'
}).reset_index()

holiday_type_analysis.columns = ['holiday_type', 'total_sales', 'days']
holiday_type_analysis['avg_daily_sales'] = holiday_type_analysis['total_sales'] / holiday_type_analysis['days']

# Calculate lift vs non-holiday baseline
non_holiday_baseline = df[df['is_holiday'] == 0]['unit_sales'].sum() / df[df['is_holiday'] == 0]['date'].nunique()

holiday_type_analysis['lift_vs_baseline'] = ((holiday_type_analysis['avg_daily_sales'] / non_holiday_baseline) - 1) * 100

# Sort by lift
holiday_type_analysis = holiday_type_analysis.sort_values('lift_vs_baseline', ascending=False)

print("\nSales by Holiday Type:")
print(holiday_type_analysis.to_string(index=False))

print(f"\nBaseline (non-holiday): {non_holiday_baseline:,.1f} units/day")

print("\nInterpretation:")
for _, row in holiday_type_analysis.iterrows():
    if row['lift_vs_baseline'] > 20:
        impact = "STRONG POSITIVE"
    elif row['lift_vs_baseline'] > 0:
        impact = "Moderate positive"
    elif row['lift_vs_baseline'] > -20:
        impact = "Slight negative"
    else:
        impact = "STRONG NEGATIVE"
    print(f"  {row['holiday_type']:<15} {row['lift_vs_baseline']:>+6.1f}%  ({impact})")

In [ ]:
# Analyze pre/post holiday effects
print("\nPre/Post Holiday Effects Analysis:")
print("=" * 70)

# Create pre/post holiday flags (±3 days around holidays)
holiday_dates = df[df['is_holiday'] == 1]['date'].unique()

df['days_to_holiday'] = df['date'].apply(
    lambda x: min([abs((x - hol).days) for hol in holiday_dates]) if len(holiday_dates) > 0 else 999
)

df['is_pre_holiday'] = ((df['days_to_holiday'] >= 1) & (df['days_to_holiday'] <= 3) & (df['is_holiday'] == 0)).astype(int)
df['is_post_holiday'] = ((df['days_to_holiday'] >= -3) & (df['days_to_holiday'] <= -1) & (df['is_holiday'] == 0)).astype(int)

# Note: This is a simplified approach - we're using absolute distance
# For better results, we'd need to track direction (before/after)

# Recalculate with proper direction
def get_holiday_proximity(date, holiday_dates):
    """
    Calculate distance and period classification for a date relative to holidays.
    
    Returns:
        tuple: (distance_in_days, period_label)
            - distance: 0 for holiday, positive for pre-holiday, negative for post-holiday, 999 for normal
            - period: 'holiday', 'pre' (1-3 days before), 'post' (1-3 days after), or 'normal'
    """
    if date in holiday_dates:
        return 0, 'holiday'
    
    future_holidays = [h for h in holiday_dates if h > date]
    past_holidays = [h for h in holiday_dates if h < date]
    
    days_to_next = min([(h - date).days for h in future_holidays]) if future_holidays else 999
    days_from_prev = min([(date - h).days for h in past_holidays]) if past_holidays else 999
    
    if days_to_next <= 3:
        return days_to_next, 'pre'
    elif days_from_prev <= 3:
        return -days_from_prev, 'post'
    else:
        return 999, 'normal'

print("Calculating holiday proximity (this may take 5 minutes)...")
df['holiday_proximity'], df['holiday_period'] = zip(*df['date'].apply(lambda x: get_holiday_proximity(x, holiday_dates)))

# Update flags
df['is_pre_holiday'] = (df['holiday_period'] == 'pre').astype(int)
df['is_post_holiday'] = (df['holiday_period'] == 'post').astype(int)

print("OK - Proximity calculated")

# Analyze by period
period_analysis = df.groupby('holiday_period').agg({
    'unit_sales': 'sum',
    'date': 'nunique'
}).reset_index()

period_analysis.columns = ['period', 'total_sales', 'days']
period_analysis['avg_daily_sales'] = period_analysis['total_sales'] / period_analysis['days']
period_analysis['lift_vs_normal'] = ((period_analysis['avg_daily_sales'] / 
                                       period_analysis[period_analysis['period'] == 'normal']['avg_daily_sales'].values[0]) - 1) * 100

print("\nSales by Holiday Period:")
period_order = ['pre', 'holiday', 'post', 'normal']
period_analysis['period'] = pd.Categorical(period_analysis['period'], categories=period_order, ordered=True)
period_analysis = period_analysis.sort_values('period')
print(period_analysis.to_string(index=False))

print("\nConclusion:")
print("  → Pre-holiday period shows shopping preparation behavior")
print("  → Holiday day itself may have lower sales (store closures)")
print("  → Post-holiday period returns to baseline")

## 2. Promotion Effectiveness Analysis

**Objective:** Measure sales lift from promotions and interactions with holidays

**Activities:**
- Compare sales: promoted vs non-promoted items
- Calculate promotion lift (% increase)
- Analyze promotion frequency by family and store
- Investigate promotion × holiday interaction
- Assess non-promoted baseline trends

**Expected output:** 
- Promotion effectiveness report (sales lift %)
- Promotion frequency analysis
- Promotion × holiday interaction insights
- ROI implications for promotional planning

In [ ]:
# Promotion effectiveness analysis
print("Promotion Effectiveness Analysis")
print("=" * 70)

print("\nPromotion distribution in dataset:")
print(f"  Total transactions: {len(df):,}")
print(f"  On promotion: {(df['onpromotion'] == 1).sum():,} ({(df['onpromotion'] == 1).sum()/len(df)*100:.2f}%)")
print(f"  Not on promotion: {(df['onpromotion'] == 0).sum():,} ({(df['onpromotion'] == 0).sum()/len(df)*100:.2f}%)")

print("\nPromotion by family:")
promo_by_family = df.groupby('family')['onpromotion'].agg(['sum', 'count'])
promo_by_family['promo_rate'] = (promo_by_family['sum'] / promo_by_family['count'] * 100)
promo_by_family.columns = ['Promoted Items', 'Total Items', 'Promotion Rate %']
print(promo_by_family)

print("\nPromotion by store type:")
promo_by_type = df.groupby('type')['onpromotion'].agg(['sum', 'count'])
promo_by_type['promo_rate'] = (promo_by_type['sum'] / promo_by_type['count'] * 100)
promo_by_type.columns = ['Promoted Items', 'Total Items', 'Promotion Rate %']
print(promo_by_type.sort_values('Promotion Rate %', ascending=False))

In [ ]:
# Calculate promotion lift
print("\nPromotion Sales Lift Analysis:")
print("=" * 70)

# Overall lift
promo_sales_avg = df[df['onpromotion'] == 1]['unit_sales'].mean()
non_promo_sales_avg = df[df['onpromotion'] == 0]['unit_sales'].mean()
overall_lift = ((promo_sales_avg / non_promo_sales_avg) - 1) * 100

print(f"\nOverall promotion effectiveness:")
print(f"  Non-promoted avg: {non_promo_sales_avg:.2f} units")
print(f"  Promoted avg: {promo_sales_avg:.2f} units")
print(f"  Sales lift: {overall_lift:+.1f}%")

# By family
print("\nPromotion lift by family:")
lift_by_family = []
for family in df['family'].unique():
    family_df = df[df['family'] == family]
    promo_avg = family_df[family_df['onpromotion'] == 1]['unit_sales'].mean()
    non_promo_avg = family_df[family_df['onpromotion'] == 0]['unit_sales'].mean()
    lift = ((promo_avg / non_promo_avg) - 1) * 100
    lift_by_family.append({
        'family': family,
        'non_promo_avg': non_promo_avg,
        'promo_avg': promo_avg,
        'lift_%': lift,
        'promo_count': (family_df['onpromotion'] == 1).sum()
    })

lift_df = pd.DataFrame(lift_by_family).sort_values('lift_%', ascending=False)
print(lift_df.to_string(index=False))

# By store type
print("\nPromotion lift by store type:")
lift_by_type = []
for store_type in df['type'].unique():
    type_df = df[df['type'] == store_type]
    promo_avg = type_df[type_df['onpromotion'] == 1]['unit_sales'].mean()
    non_promo_avg = type_df[type_df['onpromotion'] == 0]['unit_sales'].mean()
    lift = ((promo_avg / non_promo_avg) - 1) * 100
    lift_by_type.append({
        'type': store_type,
        'non_promo_avg': non_promo_avg,
        'promo_avg': promo_avg,
        'lift_%': lift,
        'promo_count': (type_df['onpromotion'] == 1).sum()
    })

lift_type_df = pd.DataFrame(lift_by_type).sort_values('lift_%', ascending=False)
print(lift_type_df.to_string(index=False))

print("\nInterpretation:")
if overall_lift > 50:
    print(f"  → Promotions are HIGHLY effective ({overall_lift:+.0f}% lift)")
elif overall_lift > 20:
    print(f"  → Promotions are effective ({overall_lift:+.0f}% lift)")
elif overall_lift > 0:
    print(f"  → Promotions have modest impact ({overall_lift:+.0f}% lift)")
else:
    print(f"  → WARNING: Promotions appear ineffective ({overall_lift:+.0f}% lift)")

In [ ]:
# Promotion × Holiday interaction
print("\nPromotion × Holiday Interaction Analysis:")
print("=" * 70)

# Create interaction categories
df['promo_holiday_category'] = 'Normal (no promo)'
df.loc[(df['onpromotion'] == 1) & (df['is_holiday'] == 0), 'promo_holiday_category'] = 'Promoted (no holiday)'
df.loc[(df['onpromotion'] == 0) & (df['is_holiday'] == 1), 'promo_holiday_category'] = 'Holiday (no promo)'
df.loc[(df['onpromotion'] == 1) & (df['is_holiday'] == 1), 'promo_holiday_category'] = 'Promoted + Holiday'

# Analyze each combination
interaction_analysis = df.groupby('promo_holiday_category').agg({
    'unit_sales': ['sum', 'mean', 'count']
}).reset_index()

interaction_analysis.columns = ['category', 'total_sales', 'avg_sales', 'count']
interaction_analysis = interaction_analysis.sort_values('avg_sales', ascending=False)

print("\nSales by Promotion × Holiday Combination:")
print(interaction_analysis.to_string(index=False))

# Calculate lifts
baseline = interaction_analysis[interaction_analysis['category'] == 'Normal (no promo)']['avg_sales'].values[0]

print(f"\nBaseline (normal days, no promo): {baseline:.2f} units")
print("\nLifts vs baseline:")
for _, row in interaction_analysis.iterrows():
    if row['category'] != 'Normal (no promo)':
        lift = ((row['avg_sales'] / baseline) - 1) * 100
        print(f"  {row['category']:<30} {lift:>+6.1f}%")

# Check for synergy (is Promo+Holiday > Promo + Holiday separately?)
promo_only_lift = ((interaction_analysis[interaction_analysis['category'] == 'Promoted (no holiday)']['avg_sales'].values[0] / baseline) - 1) * 100
holiday_only_lift = ((interaction_analysis[interaction_analysis['category'] == 'Holiday (no promo)']['avg_sales'].values[0] / baseline) - 1) * 100

if 'Promoted + Holiday' in interaction_analysis['category'].values:
    combined_lift = ((interaction_analysis[interaction_analysis['category'] == 'Promoted + Holiday']['avg_sales'].values[0] / baseline) - 1) * 100
    expected_additive = promo_only_lift + holiday_only_lift
    synergy = combined_lift - expected_additive
    
    print(f"\nSynergy Analysis:")
    print(f"  Promotion only: {promo_only_lift:>+6.1f}%")
    print(f"  Holiday only: {holiday_only_lift:>+6.1f}%")
    print(f"  Expected (additive): {expected_additive:>+6.1f}%")
    print(f"  Actual (Promo+Holiday): {combined_lift:>+6.1f}%")
    print(f"  Synergy effect: {synergy:>+6.1f}%")
    
    if synergy > 10:
        print("  → STRONG positive synergy (promotions + holidays amplify each other)")
    elif synergy > 0:
        print("  → Slight positive synergy")
    elif synergy > -10:
        print("  → Slight negative synergy (diminishing returns)")
    else:
        print("  → STRONG negative synergy (avoid combining promotions with holidays)")
else:
    print("\nNote: No 'Promoted + Holiday' combinations in sample (rare event)")

In [ ]:
# Visualize promotion effectiveness
print("\nVisualizing promotion effectiveness...")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Promotion lift by family
ax1 = axes[0]
x = np.arange(len(lift_df))
width = 0.35

bars1 = ax1.bar(x - width/2, lift_df['non_promo_avg'], width, 
                label='Non-promoted', color='#d62728', alpha=0.7)
bars2 = ax1.bar(x + width/2, lift_df['promo_avg'], width, 
                label='Promoted', color='#2ca02c', alpha=0.7)

ax1.set_ylabel('Average Unit Sales', fontsize=11)
ax1.set_title('Promotion Effectiveness by Family', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(lift_df['family'], rotation=0)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Add lift percentage labels
# Before the visualization, rename the column
lift_df = lift_df.rename(columns={'lift_%': 'lift_pct'})

# Then use the clean name
for i, row in enumerate(lift_df.itertuples()):
    ax1.text(i, max(row.non_promo_avg, row.promo_avg) + 0.5,
             f'+{row.lift_pct:.0f}%', ha='center', fontsize=10, fontweight='bold')

# Plot 2: Promotion × Holiday interaction
ax2 = axes[1]
interaction_sorted = interaction_analysis.sort_values('avg_sales')
colors_interaction = ['#7f7f7f', '#ff7f0e', '#1f77b4', '#d62728']

bars = ax2.barh(range(len(interaction_sorted)), interaction_sorted['avg_sales'],
                color=colors_interaction, alpha=0.7, edgecolor='black')

ax2.set_yticks(range(len(interaction_sorted)))
ax2.set_yticklabels(interaction_sorted['category'])
ax2.set_xlabel('Average Unit Sales', fontsize=11)
ax2.set_title('Promotion × Holiday Interaction', fontsize=13, fontweight='bold')
ax2.axvline(x=baseline, color='black', linestyle='--', alpha=0.5, 
            label=f'Baseline ({baseline:.2f})')
ax2.legend()
ax2.grid(axis='x', alpha=0.3)

# Add value labels
for i, row in enumerate(interaction_sorted.itertuples()):
    lift = ((row.avg_sales / baseline) - 1) * 100
    ax2.text(row.avg_sales, i, f' {row.avg_sales:.2f} ({lift:+.0f}%)',
             va='center', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUTS / '12_promotion_effectiveness.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nOK - Promotion visualization saved")

print("\n" + "=" * 70)
print("SECTION 2 COMPLETE - Promotion Effectiveness Analysis")
print("=" * 70)

print("\nKey Findings:")
print("  • Overall promotion lift: +74.0% (HIGHLY effective)")
print("  • BEVERAGES: +63% lift (lowest but still strong)")
print("  • CLEANING: +72% lift")
print("  • GROCERY I: +71% lift")
print("  • Type C stores: +101% lift (promotions most effective in underperforming stores)")
print("  • Negative synergy: -16.1% when combining promos with holidays")
print("  • Recommendation: Run promotions on NORMAL days, not holidays")

In [ ]:
# Perishable analysis
print("Perishable Deep Dive")
print("=" * 70)

print("\nPerishable distribution in dataset:")
print(df['perishable'].value_counts())
perishable_pct = (df['perishable'] == 1).sum() / len(df) * 100
print(f"\nPerishable transactions: {perishable_pct:.2f}%")

print("\nPerishable percentage by family:")
perishable_by_family = df.groupby('family')['perishable'].agg(['sum', 'count'])
perishable_by_family['perishable_%'] = (perishable_by_family['sum'] / perishable_by_family['count'] * 100)
perishable_by_family.columns = ['Perishable Count', 'Total', 'Perishable %']
print(perishable_by_family.sort_values('Perishable %', ascending=False))

# Sales comparison
print("\nSales comparison: Perishable vs Non-perishable:")
perishable_sales = df[df['perishable'] == 1]['unit_sales'].sum()
non_perishable_sales = df[df['perishable'] == 0]['unit_sales'].sum()
perishable_avg = df[df['perishable'] == 1]['unit_sales'].mean()
non_perishable_avg = df[df['perishable'] == 0]['unit_sales'].mean()

print(f"  Non-perishable: {non_perishable_sales:,.0f} total, {non_perishable_avg:.2f} avg")
print(f"  Perishable: {perishable_sales:,.0f} total, {perishable_avg:.2f} avg")
print(f"  Perishable avg sales: {((perishable_avg / non_perishable_avg) - 1) * 100:+.1f}% vs non-perishable")

In [ ]:
# Document the perishable limitation
print("\nPerishable Analysis Limitation:")
print("=" * 70)

print("\nFINDING: Our sample contains 0% perishable items")
print("\nExplanation:")
print("  • Top-3 families selected (GROCERY I, BEVERAGES, CLEANING) are non-perishable")
print("  • Perishable items concentrated in other families:")
print("    - PRODUCE (fresh fruits, vegetables)")
print("    - DAIRY (milk, cheese, yogurt)")
print("    - MEATS (fresh meat, seafood)")
print("    - BREAD/BAKERY (fresh baked goods)")
print("    - DELI (prepared foods)")

print("\nImplication for forecasting:")
print("  → Our models will focus on non-perishable inventory")
print("  → Lower forecasting accuracy requirements (less waste risk)")
print("  → Longer shelf life = more forecast horizon flexibility")
print("  → Safety stock strategies different from perishables")

print("\nFor complete perishable analysis:")
print("  → Would need to include PRODUCE, DAIRY, MEATS in sample")
print("  → Noted as limitation in project documentation")

# Check actual items.csv to confirm perishable distribution
print("\nVerifying against full items.csv...")
df_items_full = pd.read_csv(DATA_RAW / 'items.csv')
print(f"\nFull item catalog:")
print(f"  Total items: {len(df_items_full):,}")
print(f"  Perishable items: {(df_items_full['perishable'] == 1).sum():,} ({(df_items_full['perishable'] == 1).sum()/len(df_items_full)*100:.1f}%)")
print(f"  Non-perishable items: {(df_items_full['perishable'] == 0).sum():,} ({(df_items_full['perishable'] == 0).sum()/len(df_items_full)*100:.1f}%)")

print("\nPerishable items by family (full catalog):")
perishable_full = df_items_full.groupby('family')['perishable'].agg(['sum', 'count'])
perishable_full['perishable_%'] = (perishable_full['sum'] / perishable_full['count'] * 100)
perishable_full.columns = ['Perishable Count', 'Total Items', 'Perishable %']
perishable_full = perishable_full[perishable_full['Perishable Count'] > 0].sort_values('Perishable %', ascending=False)
print(perishable_full.head(10))

print("\nOur top-3 families in full catalog:")
our_families = df_items_full[df_items_full['family'].isin(['GROCERY I', 'BEVERAGES', 'CLEANING'])]
print(f"  Total items in top-3: {len(our_families):,}")
print(f"  Perishable in top-3: {(our_families['perishable'] == 1).sum()} ({(our_families['perishable'] == 1).sum()/len(our_families)*100:.2f}%)")

print("\n" + "=" * 70)
print("SECTION 3 COMPLETE - Perishable Analysis (Scope Limitation Noted)")
print("=" * 70)

print("\nKey Finding:")
print("  • 0% perishable items in our sample (by design - top-3 families are non-perishable)")
print("  • Project focus: Non-perishable forecasting (longer shelf life, lower waste risk)")
print("  • Limitation documented for stakeholder communication")

## 4. External Factors & Zero-Sales Analysis

**Objective:** Investigate oil price correlation and zero-sales patterns

**Activities:**
- Load oil.csv and merge with aggregated daily sales
- Plot dual-axis chart (oil price + sales over time)
- Calculate correlation coefficient
- Analyze zero-sales patterns (true zeros vs stockouts)
- Document findings for feature selection

**Expected output:** 
- Oil price vs sales correlation analysis
- Zero-sales pattern report
- Decision on oil feature inclusion (Week 2)

In [ ]:
# Oil price analysis
print("External Factors Analysis")
print("=" * 70)

print("\n1. Oil Price Correlation:")
print("=" * 70)

print("\nLoading oil.csv...")
df_oil = pd.read_csv(DATA_RAW / 'oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])

print(f"OK - Oil data loaded")
print(f"  Records: {len(df_oil):,}")
print(f"  Date range: {df_oil['date'].min().date()} to {df_oil['date'].max().date()}")
print(f"  Missing values: {df_oil['dcoilwtico'].isnull().sum()} ({df_oil['dcoilwtico'].isnull().sum()/len(df_oil)*100:.1f}%)")

print("\nOil price statistics:")
print(df_oil['dcoilwtico'].describe())

# Aggregate daily sales for correlation
print("\nAggregating daily sales...")
daily_sales = df.groupby('date')['unit_sales'].sum().reset_index()
daily_sales.columns = ['date', 'total_sales']

print(f"  Daily sales records: {len(daily_sales):,}")

# Merge oil with sales
print("\nMerging oil prices with daily sales...")
df_oil_sales = daily_sales.merge(df_oil, on='date', how='left')

print(f"  Merged records: {len(df_oil_sales):,}")
print(f"  Missing oil prices after merge: {df_oil_sales['dcoilwtico'].isnull().sum()}")

# Forward fill missing oil prices
df_oil_sales['dcoilwtico'] = df_oil_sales['dcoilwtico'].fillna(method='ffill')

# Calculate correlation
correlation = df_oil_sales[['total_sales', 'dcoilwtico']].corr().iloc[0, 1]

print(f"\nCorrelation Analysis:")
print(f"  Pearson correlation: {correlation:.4f}")

if abs(correlation) > 0.6:
    strength = "Strong"
elif abs(correlation) > 0.3:
    strength = "Moderate"
else:
    strength = "Weak"

print(f"  Interpretation: {strength} {'positive' if correlation > 0 else 'negative'} correlation")

In [ ]:
# Visualize oil price vs sales
print("\nVisualizing oil price vs sales relationship...")

fig, ax1 = plt.subplots(figsize=(16, 6))

# Plot sales on left axis
color = 'steelblue'
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Total Daily Sales (units)', color=color, fontsize=12)
ax1.plot(df_oil_sales['date'], df_oil_sales['total_sales'], 
         color=color, linewidth=1, alpha=0.6, label='Daily Sales')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True, alpha=0.3)

# Plot oil price on right axis
ax2 = ax1.twinx()
color = 'darkred'
ax2.set_ylabel('Oil Price (WTI, USD)', color=color, fontsize=12)
ax2.plot(df_oil_sales['date'], df_oil_sales['dcoilwtico'], 
         color=color, linewidth=2, alpha=0.7, label='Oil Price')
ax2.tick_params(axis='y', labelcolor=color)

# Title and legend
plt.title(f'Oil Price vs Sales (Correlation: {correlation:.3f})', 
          fontsize=14, fontweight='bold')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.tight_layout()
plt.savefig(OUTPUTS / '13_oil_price_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nOK - Oil price visualization saved")

print("\nInterpretation:")
print("  • Moderate negative correlation (-0.55)")
print("  • When oil prices rise → sales tend to fall")
print("  • Ecuador is oil-dependent: high oil prices → inflation → reduced purchasing power")
print("  • This correlation is STRONG ENOUGH to include as a predictive feature")
print("  • Notable periods:")
print("    - 2014-2015: Oil price crash (~$110 → $26) coincides with sales increase")
print("    - 2016-2017: Oil price recovery (~$26 → $50) shows sales stabilization")

print("\nDecision for Week 2 (Feature Engineering):")
if abs(correlation) > 0.3:
    print(f"  → INCLUDE oil price as feature (strong/moderate correlation: {correlation:.3f})")
    print("  → Create lagged oil price features (7/14/30-day)")
    print("  → Consider oil price change (derivative) as feature")
else:
    print(f"  → EXCLUDE oil price as feature (weak correlation: {correlation:.3f})")

In [ ]:
# Zero-sales analysis
print("\n2. Zero-Sales Pattern Analysis:")
print("=" * 70)

print("\nAnalyzing zero-sales transactions...")
zero_sales = df[df['unit_sales'] == 0]
non_zero_sales = df[df['unit_sales'] > 0]

print(f"  Total transactions: {len(df):,}")
print(f"  Zero sales: {len(zero_sales):,} ({len(zero_sales)/len(df)*100:.2f}%)")
print(f"  Non-zero sales: {len(non_zero_sales):,} ({len(non_zero_sales)/len(df)*100:.2f}%)")

print("\nZero-sales by family:")
zero_by_family = df.groupby('family')['unit_sales'].apply(lambda x: (x == 0).sum()).sort_values(ascending=False)
total_by_family = df.groupby('family').size()
zero_pct_family = (zero_by_family / total_by_family * 100).round(2)

for family in zero_by_family.index:
    print(f"  {family:<15} {zero_by_family[family]:>6,} ({zero_pct_family[family]:>5.2f}%)")

print("\nInterpretation:")
print("  • Sparse data contains implicit zeros (store-item-date combinations not in dataset)")
print("  • Explicit zeros (0.30%): Likely data recording or returns processing")
print("  • Our 99.1% sparsity (from Day 3) represents true zero demand days")
print("  • Zero-sales distinction:")
print("    - Explicit zeros: In dataset with unit_sales=0")
print("    - Implicit zeros: Missing from dataset (store-item didn't sell that day)")

print("\nImplication for forecasting:")
print("  → Models must handle sparse time series")
print("  → Zero-inflation models may be appropriate")
print("  → Feature: 'days_since_last_sale' could be valuable")

print("\n" + "=" * 70)
print("SECTION 4 COMPLETE - External Factors & Zero-Sales")
print("=" * 70)

print("\nKey Findings:")
print("  • Oil price: -0.55 correlation (INCLUDE as feature)")
print("  • Zero-sales: 0.30% explicit, 99.1% sparsity (implicit zeros)")
print("  • Recommendation: Use sparse time series models")


## 5. Export & Documentation

**Objective:** Save final analysis-ready dataset and create Week 1 summary

**Activities:**
- Export final dataset to CSV and pickle (guayas_prepared.csv/.pkl)
- Create feature dictionary documenting all columns
- Update decision log with Week 1 decisions
- Generate Week 1 summary report
- Final notebook completion summary

**Expected output:** 
- guayas_prepared.csv (analysis-ready dataset)
- guayas_prepared.pkl (fast-loading format)
- Feature dictionary
- Week 1 summary report
- Decision log updated

In [ ]:
# Export final dataset
print("Final Dataset Export")
print("=" * 70)

print("\nPreparing final dataset for export...")

# Final dataset characteristics
print(f"\nFinal dataset summary:")
print(f"  Rows: {len(df):,}")
print(f"  Columns: {len(df.columns)}")
print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"  Date range: {df['date'].min().date()} to {df['date'].max().date()}")
print(f"  Missing values: {df.isnull().sum().sum()}")

print("\nColumn list:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:>2}. {col}")

# Export to CSV
csv_path = DATA_PROCESSED / 'guayas_prepared.csv'
print(f"\nExporting to CSV: {csv_path.name}")
df.to_csv(csv_path, index=False)
csv_size = csv_path.stat().st_size / 1024**2
print(f"  ✓ CSV saved ({csv_size:.1f} MB)")

# Export to pickle
pkl_path = DATA_PROCESSED / 'guayas_prepared.pkl'
print(f"\nExporting to pickle: {pkl_path.name}")
df.to_pickle(pkl_path)
pkl_size = pkl_path.stat().st_size / 1024**2
print(f"  ✓ Pickle saved ({pkl_size:.1f} MB)")

print("\nOK - Final dataset exported")
print(f"  CSV: {csv_path.resolve()}")
print(f"  Pickle: {pkl_path.resolve()}")

In [ ]:
# Create feature dictionary
print("\nCreating Feature Dictionary...")
print("=" * 70)

feature_dict = {
    # Original features
    'id': 'Unique transaction ID from Kaggle dataset',
    'date': 'Transaction date (YYYY-MM-DD)',
    'store_nbr': 'Store number (24-51 in Guayas region)',
    'item_nbr': 'Item/product number (unique SKU identifier)',
    'unit_sales': 'Number of units sold (target variable)',
    'onpromotion': 'Promotion flag (1=promoted, 0=not promoted)',
    'family': 'Product family (GROCERY I, BEVERAGES, CLEANING)',
    'class': 'Product class (subcategory within family)',
    'perishable': 'Perishable flag (0=non-perishable, 1=perishable) - All 0 in sample',
    
    # Store metadata
    'city': 'Store city (Guayaquil, Daule, Libertad)',
    'state': 'Store state (Guayas)',
    'type': 'Store type (A=premium, B=good, C=medium, D=basic, E=entry)',
    'cluster': 'Store cluster (1-17, grouping similar stores)',
    
    # Temporal features (Day 3)
    'year': 'Year (2013-2017)',
    'month': 'Month (1-12)',
    'day': 'Day of month (1-31)',
    'day_of_week': 'Day of week (0=Monday, 6=Sunday)',
    'day_of_month': 'Day within month (1-31, duplicate of "day")',
    'is_weekend': 'Weekend flag (1=Sat/Sun, 0=weekday)',
    
    # Holiday features (Day 5)
    'is_holiday': 'Holiday flag (1=national holiday, 0=normal day)',
    'holiday_type': 'Holiday type (Holiday, Event, Additional, Transfer, Work Day, Bridge)',
    'holiday_name': 'Holiday description (e.g., Christmas, Independence Day)',
    'days_to_holiday': 'Days to nearest holiday (absolute distance)',
    'is_pre_holiday': 'Pre-holiday flag (1-3 days before holiday)',
    'is_post_holiday': 'Post-holiday flag (1-3 days after holiday)',
    'holiday_proximity': 'Signed days to holiday (negative=after, positive=before)',
    'holiday_period': 'Holiday period label (pre, holiday, post, normal)',
    'promo_holiday_category': 'Promotion × Holiday interaction category'
}

print("\nFeature Dictionary (28 features):")
print("\n" + "=" * 70)
for i, (feature, description) in enumerate(feature_dict.items(), 1):
    print(f"{i:>2}. {feature:<25} {description}")

print("\n" + "=" * 70)
print("Feature Categories:")
print("  • Original (9): id, date, store_nbr, item_nbr, unit_sales, onpromotion, family, class, perishable")
print("  • Store metadata (4): city, state, type, cluster")
print("  • Temporal (6): year, month, day, day_of_week, day_of_month, is_weekend")
print("  • Holiday (9): is_holiday, holiday_type, holiday_name, days_to_holiday, is_pre_holiday,")
print("                 is_post_holiday, holiday_proximity, holiday_period, promo_holiday_category")

print("\nFeatures NOT YET created (Week 2):")
print("  • Rolling statistics (7/14/30-day moving averages)")
print("  • Lag features (1/7/14/30-day lags)")
print("  • Oil price features (daily price, 7/14/30-day lags)")
print("  • Store/item aggregations (store avg, item avg)")
print("  • Promotion history (promotion frequency, days since last promo)")

# Save feature dictionary
dict_path = project_root / 'docs' / 'feature_dictionary.txt'
with open(dict_path, 'w') as f:
    f.write("Feature Dictionary - Guayas Prepared Dataset\n")
    f.write("=" * 70 + "\n\n")
    for feature, description in feature_dict.items():
        f.write(f"{feature:<25} {description}\n")

print(f"\n✓ Feature dictionary saved: {dict_path.resolve()}")

In [ ]:
# Week 1 completion summary
print("\n" + "=" * 70)
print("DAY 5 COMPLETE - Context Analysis & Export")
print("=" * 70)

print("\nDay 5 Summary:")
print("  ✓ Holiday impact analysis (+24.2% overall, +49.6% Additional days)")
print("  ✓ Promotion effectiveness (+74% lift, Type C stores +101%)")
print("  ✓ Promotion × Holiday synergy (-16.1% negative, avoid combining)")
print("  ✓ Perishable analysis (0% in sample, scope limitation documented)")
print("  ✓ Oil price correlation (-0.55 moderate, INCLUDE as feature)")
print("  ✓ Zero-sales patterns (99.1% sparsity, implicit zeros)")
print("  ✓ Final dataset exported (guayas_prepared.csv, 300K rows, 28 cols)")
print("  ✓ Feature dictionary created")

print("\n" + "=" * 70)
print("WEEK 1 COMPLETE - Exploration & Understanding")
print("=" * 70)

print("\n WEEK 1 ACCOMPLISHMENTS:")
print("\n1. Data Scope & Quality:")
print("   • 11 Guayas stores analyzed (73.8% in Guayaquil)")
print("   • Top-3 families: GROCERY I, BEVERAGES, CLEANING")
print("   • 300K sample (representative, manageable)")
print("   • 0% missing values after cleaning")
print("   • 0.28% high-confidence outliers (retained)")
print("   • 99.1% sparsity (retail reality documented)")

print("\n2. Store Performance Insights:")
print("   • 4.25x performance gap (Store #51: 356K vs Store #32: 84K)")
print("   • Type A stores: 2x higher avg sales vs Type C")
print("   • 49% universal items (sold in all stores)")
print("   • Item coverage: 64.7% to 89.9% range")

print("\n3. Temporal Patterns:")
print("   • Weekend lift: +33.9% (BEVERAGES +40.2%)")
print("   • Payday effect: +10.7% (Day 1 peak at +21.9%)")
print("   • December seasonality: +30.4%")
print("   • Thursday lowest day (78-84% of average)")
print("   • Strong autocorrelation: 0.32-0.63 at lags 1-90")

print("\n4. Product Dynamics:")
print("   • Pareto: 34% items = 80% sales")
print("   • Fast movers (20%): 58.4% of sales")
print("   • Slow movers (20%): 2.2% of sales")
print("   • Top item velocity: 70.21 units/day")

print("\n5. Context Factors:")
print("   • Holiday lift: +24.2% overall")
print("   • Additional days: +49.6% (highest)")
print("   • Event days: +24.7%")
print("   • Regular holidays: -0.4% (closures)")
print("   • Promotion lift: +74% (HIGHLY effective)")
print("   • Type C stores: +101% promo lift")
print("   • Promo × Holiday: -16.1% synergy (avoid)")
print("   • Oil correlation: -0.55 (moderate negative)")

print("\n6. Decisions Logged (7 total):")
print("   • DEC-001: Top-3 families by item count")
print("   • DEC-002: 300K sample for development speed")
print("   • DEC-003: Fill onpromotion NaN with False")
print("   • DEC-004: 3-method outlier detection (retain)")
print("   • DEC-005: Keep sparse format (no gap filling)")
print("   • DEC-006: Rolling stats with min_periods=1")
print("   • DEC-007: Fast/slow velocity classification")

print("\n7. Deliverables:")
print("   • 5 notebooks: d01-d05 (setup, sampling, quality, temporal, context)")
print("   • 13 visualizations: store perf, outliers, time series, holidays, promos")
print("   • Final dataset: guayas_prepared.csv (300K × 28)")
print("   • Feature dictionary: 28 features documented")
print("   • 2 checkpoints: Day 3, Day 4")

print("\n" + "=" * 70)
print("WEEK 2 PREVIEW - Feature Development")
print("=" * 70)

print("\nWeek 2 Priorities:")
print("  1. Lag features (1/7/14/30-day)")
print("  2. Rolling statistics (already started: 7/14/30-day)")
print("  3. Oil price features (daily, 7/14/30-day lags)")
print("  4. Store/item aggregations (historical averages)")
print("  5. Promotion history (frequency, days since last)")
print("  6. Holiday proximity features (optimize window)")
print("  7. Feature importance analysis (Week 3)")

print("\nKey Recommendations:")
print("  → Run promotions on NORMAL days (not holidays)")
print("  → Elevate weekend inventory 30-40%")
print("  → Focus forecasting on fast movers (34% items)")
print("  → Include oil price as macro indicator")
print("  → Use sparse time series models")
print("  → Type C stores need promotional support")

print("\n" + "=" * 70)
print("TIME SUMMARY")
print("=" * 70)

time_summary = {
    'Day 1': {'allocated': 4.0, 'actual': 3.0, 'status': '✓'},
    'Day 2': {'allocated': 4.0, 'actual': 1.5, 'status': '✓'},
    'Day 3': {'allocated': 5.5, 'actual': 4.5, 'status': '✓'},
    'Day 4': {'allocated': 5.0, 'actual': 3.5, 'status': '✓'},
    'Day 5': {'allocated': 5.0, 'actual': 2.5, 'status': '✓'}
}

print("\nWeek 1 Time Tracking:")
total_allocated = sum(d['allocated'] for d in time_summary.values())
total_actual = sum(d['actual'] for d in time_summary.values())
buffer = total_allocated - total_actual

for day, times in time_summary.items():
    variance = times['allocated'] - times['actual']
    print(f"  {day}: {times['actual']:.1f}h / {times['allocated']:.1f}h ({variance:+.1f}h) {times['status']}")

print(f"\n  Total: {total_actual:.1f}h / {total_allocated:.1f}h")
print(f"  Buffer remaining: {buffer:.1f}h")
print(f"  Efficiency: {(total_actual/total_allocated)*100:.0f}%")

print("\n WEEK 1 COMPLETE - EXCELLENT PROGRESS!")
print(f"\nCompleted {buffer:.1f} hours ahead of schedule!")
print("Ready for Week 2: Feature Development")